# Classify particle motion from MSD anaysis and bayesian inference (in construction)

This analysis is largely inspired from the following paper _Monnier, N. (2012). Bayesian Approach to MSD-Based Analysis of Particle Motion in Live Cells. Biophysical Journal_.

The idea is to classify particle motion in different biophysical model : diffusion, confined movement, direct, and so forth.

The input of the analysis is MSD curves of several particles (under same condition) and the output is a set of probability for different models.

For more details, the papier is available here : http://www.cell.com/biophysj/abstract/S0006-3495(12)00718-7

For a complete introduction of bayesian statistic, I strongly encourage you to read this excellent book : [Bayesian Methods for Hackers](http://nbviewer.ipython.org/github/CamDavidsonPilon/Probabilistic-Programming-and-Bayesian-Methods-for-Hackers/blob/master/Chapter1_Introduction/Chapter1.ipynb).

TODO: introduce the theory

![model](model.png)

_Monnier, N. (2012). Bayesian Approach to MSD-Based Analysis of Particle Motion in Live Cells. Biophysical Journal_

In [1]:
%matplotlib qt
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
from scipy import io

import pymc3 as pm
import theano
import theano.tensor as t

import matplotlib.pyplot as plt

## Load chromosomes MSD curves

Corresponds to Fig. 4 A-C in the paper.

![chromosomes](chromosomes.png)

_Monnier, N. (2012). Bayesian Approach to MSD-Based Analysis of Particle Motion in Live Cells. Biophysical Journal_

In [2]:
# Chromosomes traj
mat = io.loadmat('chromosomes.mat')
msds = mat['MSD_curves_chromosomes']
msds = pd.DataFrame(msds)
msds["delay"] = mat['timelags'].T[0]
msds.set_index("delay", drop=True, inplace=True)
msds.head()

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
delay,,,,,,,,,,,,,,,,,,,,,
2.6,0.097573,0.373977,0.076612,0.068739,0.062353,0.036330,0.153783,0.099404,0.204890,0.292111,...,0.161717,0.132908,0.276165,0.134094,0.178738,0.175778,0.256930,0.169733,0.117894,0.262046
5.2,0.196207,0.351178,0.128585,0.123823,0.147403,0.061929,0.301612,0.227539,0.464767,0.573292,...,0.405083,0.334022,0.662523,0.338944,0.434863,0.418021,0.650792,0.432889,0.309397,0.589823
7.8,0.331055,0.470176,0.176194,0.198717,0.260169,0.092897,0.485697,0.379726,0.783481,0.893782,...,0.726817,0.593216,1.095236,0.588966,0.738894,0.725592,1.128657,0.770092,0.555813,0.947340
10.4,0.464399,0.473161,0.232980,0.281506,0.388835,0.133016,0.682776,0.554917,1.164619,1.313681,...,1.115730,0.932779,1.593391,0.893019,1.100812,1.095992,1.713535,1.185670,0.852484,1.355115
13.0,0.588834,0.479392,0.276361,0.364231,0.531622,0.175939,0.913439,0.759349,1.602612,1.764976,...,1.624899,1.348220,2.226334,1.278424,1.561110,1.566704,2.423151,1.691124,1.214357,1.832992


Display all the MSD curves

In [3]:
fig, ax = plt.subplots(figsize=(10, 8))
msds.plot(ax=ax, legend=False)

ax.set_xlabel('Delay (s)')
ax.set_ylabel('MSD ($\mu m^2.s^{-1}$)')

Display the average MSD (with std and sem)

In [4]:
msd_mean = msds.mean(axis=1)
msd_std = msds.std(axis=1)
msd_sem = msds.sem(axis=1)

fig, ax = plt.subplots(figsize=(10, 8))
msd_mean.plot(ax=ax, lw=2)

# std
ax.fill_between(msd_mean.index, msd_mean, msd_mean + msd_std, alpha=0.1)
ax.fill_between(msd_mean.index, msd_mean, msd_mean - msd_std, alpha=0.1)

# sem
ax.fill_between(msd_mean.index, msd_mean, msd_mean + msd_sem, alpha=0.2)
ax.fill_between(msd_mean.index, msd_mean, msd_mean - msd_sem, alpha=0.2)

ax.set_xlabel('Delay (s)')
ax.set_ylabel('MSD ($\mu m^2.s^{-1}$)')

## Naive implementation from Matlab code

Matlab code is available here : http://msd-bayes.org/

### Covariance matrix

In `msd_curves_bayes.m`.

In [5]:
# Get the average MSD
msd_mean = msds.mean(axis=1)

# Get difference between each individual curve and the mean curve
errors = msds.copy()
for i, col in msds.iteritems():
    errors.loc[:, i] = col - msd_mean

# Calculate raw covariance matrix
error_cov_raw = np.cov(errors)

# Regularize covariance matrix (TODO)
error_cov = error_cov_raw.copy()

# Covariance of the mean curve
error_cov_raw /= errors.shape[0]
error_cov /= errors.shape[0]

Display the covariance matrix.

In [6]:
plt.figure(figsize=(8, 8))
plt.imshow(error_cov)

### Fitting

In `msd_fitting.m`.

In [ ]:
# TODO

## Implementation with PyMC3

See https://pymc-devs.github.io/pymc3/getting_started/#a-motivating-example-linear-regression for an introduction to PyMC3.

### Brownian diffusion

Fit the following equation : $MSD(\tau) = 4D\tau$

** IN CONSTRUCTION **
based on http://nbviewer.ipython.org/github/CamDavidsonPilon/Probabilistic-Programming-and-Bayesian-Methods-for-Hackers/blob/master/Chapter1_Introduction/Chapter1.ipynb#Introducing-our-first-hammer:-PyMC

In [44]:
count_data = np.loadtxt("/home/hadim/txtdata.csv")
plt.bar(np.arange(len(count_data)), count_data, color="#348ABD", edgecolor="none")

<Container object of 74 artists>

In [54]:
count_data = np.loadtxt("/home/hadim/txtdata.csv")

alpha = 1.0 / count_data.mean()  # Recall count_data is the variable that holds our txt counts

with pm.Model() as model:

    lambda_1 = pm.Exponential("lambda_1", alpha, testval=17)
    lambda_2 = pm.Exponential("lambda_2", alpha)

    tau = pm.DiscreteUniform("tau", lower=0, upper=len(count_data))

    days = np.arange(len(count_data))
    lambda_ = pm.switch(tau >= days, lambda_1, lambda_2)
    
    observation = pm.Poisson("observation", mu=lambda_, observed=count_data)
    
    step = pm.Metropolis()
    trace = pm.sample(1000, step)
    
print()
print("tau", trace['tau'].mean())
print("lambda_1", trace['lambda_1'].mean())
print("lambda_2", trace['lambda_2'].mean())

 [-----------------100%-----------------] 1000 of 1000 complete in 0.4 sec
tau 42.181
lambda_1 18.1154265182
lambda_2 22.2406555915


In [33]:
pm.traceplot(trace)

array([[<matplotlib.axes._subplots.AxesSubplot object at 0x7f199d0ea198>,
        <matplotlib.axes._subplots.AxesSubplot object at 0x7f199cb45748>]], dtype=object)

In [55]:
tau = trace['tau'].mean()
lambda_1 = trace['lambda_1'].mean()
lambda_2 = trace['lambda_2'].mean()

mcount = np.zeros(count_data.shape)
mcount[:tau] = lambda_1
mcount[tau:] = lambda_2

plt.figure(figsize=(10, 6))
count_data = np.loadtxt("/home/hadim/txtdata.csv")
plt.bar(np.arange(len(count_data)), count_data, color="#348ABD", edgecolor="none")
plt.plot(mcount, lw=4, color="#E24A33")

In [56]:
count_data = np.loadtxt("/home/hadim/txtdata.csv")

@theano.compile.ops.as_op(itypes=[t.lscalar, t.lscalar, t.dscalar, t.dscalar, t.dscalar], otypes=[t.dvector])
def lambda_(tau1, tau2, lambda_1, lambda_2, lambda_3):
    out = np.zeros(len(count_data))
    out[:tau1] = lambda_1  # lambda before tau is lambda1
    out[tau1:tau2] = lambda_2  # lambda before tau is lambda1
    out[tau2:] = lambda_3  # lambda after (and including) tau is lambda2
    return out

alpha = 1.0 / count_data.mean()  # Recall count_data is the variable that holds our txt counts

with pm.Model() as model:

    lambda_1 = pm.Exponential("lambda_1", alpha)
    lambda_2 = pm.Exponential("lambda_2", alpha)
    lambda_3 = pm.Exponential("lambda_3", alpha)

    tau1 = pm.DiscreteUniform("tau1", lower=0, upper=len(count_data))
    tau2 = pm.DiscreteUniform("tau2", lower=0, upper=len(count_data))
    
    observation = pm.Poisson("observation", mu=lambda_(tau1, tau2, lambda_1, lambda_2, lambda_3),
                             observed=count_data)
    
    step = pm.Metropolis()
    trace = pm.sample(5000, step)
    
print()
print("tau1", trace['tau1'].mean())
print("tau2", trace['tau2'].mean())

print("lambda_1", trace['lambda_1'].mean())
print("lambda_2", trace['lambda_2'].mean())
print("lambda_3", trace['lambda_3'].mean())

 [-----------------100%-----------------] 5000 of 5000 complete in 4.1 sec
tau1 31.988
tau2 43.5846
lambda_1 20.1536361894
lambda_2 11.2837949963
lambda_3 22.5687064043


In [41]:
pm.traceplot(trace)

array([[<matplotlib.axes._subplots.AxesSubplot object at 0x7f199795b0f0>,
        <matplotlib.axes._subplots.AxesSubplot object at 0x7f19970cacc0>]], dtype=object)

In [57]:
tau1 = trace['tau1'].mean()
tau2 = trace['tau2'].mean()
lambda_1 = trace['lambda_1'].mean()
lambda_2 = trace['lambda_2'].mean()
lambda_3 = trace['lambda_3'].mean()

mcount = np.zeros(count_data.shape)
mcount[:tau1] = lambda_1
mcount[tau1:tau2] = lambda_2
mcount[tau2:] = lambda_3

plt.figure(figsize=(10, 6))
count_data = np.loadtxt("/home/hadim/txtdata.csv")
plt.bar(np.arange(len(count_data)), count_data, color="#348ABD", edgecolor="none")
plt.plot(mcount, lw=4, color="#E24A33")